In [1]:
import pandas as pd

In [7]:
data = pd.read_csv('horse_data.csv', na_values = '?').iloc[:,[1, 3, 4, 10, 12, 17, 19, 26]]
data.columns = [
    'age',
    'rectal temperature',
    'pulse',
    'pain',
    'abdominal distension',
    'abdomen',
    'total protein',
    'type of surgical lesion'
]
data

,age,rectal temperature,pulse,pain,abdominal distension,abdomen,total protein,type of surgical lesion
0,1,39.2,88.0,3.0,2.0,2.0,85.0,0
1,1,38.3,40.0,3.0,1.0,1.0,6.7,0
2,9,39.1,164.0,2.0,4.0,NaN,7.2,0
3,1,37.3,104.0,NaN,NaN,NaN,7.4,0
4,1,NaN,NaN,2.0,2.0,3.0,NaN,0
...,...,...,...,...,...,...,...,...
294,1,NaN,120.0,2.0,NaN,5.0,65.0,0
295,1,37.2,72.0,4.0,3.0,4.0,NaN,0
296,1,37.5,72.0,4.0,3.0,5.0,6.8,0
297,1,36.5,100.0,3.0,3.0,4.0,6.0,0


In [8]:
data.describe()

,age,rectal temperature,pulse,pain,abdominal distension,abdomen,total protein,type of surgical lesion
count,299.00000,239.000000,275.000000,244.000000,243.000000,181.000000,266.000000,299.000000
mean,1.64214,38.166527,71.934545,2.942623,2.259259,3.685083,24.517293,7.387960
std,2.17730,0.733508,28.680522,1.303993,1.061472,1.492667,27.509056,127.749768
min,1.00000,35.400000,30.000000,1.000000,1.000000,1.000000,3.300000,0.000000
25%,1.00000,37.800000,48.000000,2.000000,1.000000,2.000000,6.500000,0.000000
50%,1.00000,38.200000,64.000000,3.000000,2.000000,4.000000,7.500000,0.000000
75%,1.00000,38.500000,88.000000,4.000000,3.000000,5.000000,57.000000,0.000000
max,9.00000,40.800000,184.000000,5.000000,4.000000,5.000000,89.000000,2209.000000


In [9]:
data.median()

age                         1.0
rectal temperature         38.2
pulse                      64.0
pain                        3.0
abdominal distension        2.0
abdomen                     4.0
total protein               7.5
type of surgical lesion     0.0
dtype: float64

In [10]:
data.round().mode()

,age,rectal temperature,pulse,pain,abdominal distension,abdomen,total protein,type of surgical lesion
0,1,38.0,48.0,3.0,1.0,5.0,7.0,0


Вкратце, в среднем температура чуть выше средней (норма 37.8). Можно сказать, что большая часть пациентов все же имеет некоторые осложнения. Пульс повышенный (норма 30-40). Боль, в основном, легкая прерывистая. Вздутие живота не было, хотя у некоторых слабое все же есть. У большинства брюшная полость растянута в толстом кишечнике. Уровень белка у пациентов в норме (норма 6-7.5). До операций дело не дошло. 



Видно, что есть пациент, а может даже несколько, который явно старше остальных (9 месяцев), но это значение нельзя назвать выбросом, потому что похоже на естественные данные. Эти данные не игнорируем, их отсутствие даст неправильную статистику!

А вот с количеством белка точно что-то не так. Есть значение 89 - максимальное, и минимальное - 3. Скорее всего значения выше 15 будут выбросом, потому что такие невозможны. А около 3, я думаю, реальны, поэтому минимум за выброс не берем. Возможно, выбросы тут преднамеренные или из-за ошибки ввода данных. Чтобы сильно не искажать реальные данные, можно попробовать заменить выбросы на медиану.

Аналогично с типом операции (столбец 'type of surgical lesion') видно выбросы. Максимальное значение таблицы > 2000, а по условию - 11. Поэтому все, что выше 11, будем считать выбросом. Возможно, выбросы тут преднамеренные или из-за ошибки ввода данных. Т.к. значения дискретные, выбросы заменим модой.

In [11]:
# заменим выбросы на медиану в столбце total protein
data.loc[data['total protein'] > 15, 'total protein'] = data['total protein'].median()

In [12]:
# заменим выбросы на моду в столбце type of surgical lesion
data.loc[data['type of surgical lesion'] > 11, 'type of surgical lesion'] = data['type of surgical lesion'].mode()

In [13]:
# новые средние значения
data.mean().round()

age                         2.0
rectal temperature         38.0
pulse                      72.0
pain                        3.0
abdominal distension        2.0
abdomen                     4.0
total protein               7.0
type of surgical lesion     0.0
dtype: float64

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      299 non-null    int64  
 1   rectal temperature       239 non-null    float64
 2   pulse                    275 non-null    float64
 3   pain                     244 non-null    float64
 4   abdominal distension     243 non-null    float64
 5   abdomen                  181 non-null    float64
 6   total protein            266 non-null    float64
 7   type of surgical lesion  298 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.8 KB


In [15]:
# количество пропусков 
for col in data.columns:
    pct_missing = data[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

age - 0.0%
rectal temperature - 20.1%
pulse - 8.0%
pain - 18.4%
abdominal distension - 18.7%
abdomen - 39.5%
total protein - 11.0%
type of surgical lesion - 0.3%


In [16]:
data.describe()

,age,rectal temperature,pulse,pain,abdominal distension,abdomen,total protein,type of surgical lesion
count,299.00000,239.000000,275.000000,244.000000,243.000000,181.000000,266.000000,298.0
mean,1.64214,38.166527,71.934545,2.942623,2.259259,3.685083,7.113158,0.0
std,2.17730,0.733508,28.680522,1.303993,1.061472,1.492667,1.000347,0.0
min,1.00000,35.400000,30.000000,1.000000,1.000000,1.000000,3.300000,0.0
25%,1.00000,37.800000,48.000000,2.000000,1.000000,2.000000,6.500000,0.0
50%,1.00000,38.200000,64.000000,3.000000,2.000000,4.000000,7.500000,0.0
75%,1.00000,38.500000,88.000000,4.000000,3.000000,5.000000,7.500000,0.0
max,9.00000,40.800000,184.000000,5.000000,4.000000,5.000000,13.000000,0.0


Пропуски заменим медианным значением, чтобы не исказить данные

In [17]:
data['rectal temperature'].fillna(data['rectal temperature'].median(), inplace=True)
data['pulse'].fillna(data['pulse'].median(), inplace = True)
data['pain'].fillna(data['pain'].median(), inplace=True)
data['abdominal distension'].fillna(data['abdominal distension'].median(), inplace=True)
data['total protein'].fillna(data['total protein'].median(), inplace=True)
data['abdomen'].fillna(data['abdomen'].median(), inplace=True)
data['type of surgical lesion'].fillna(data['type of surgical lesion'].median(), inplace=True)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      299 non-null    int64  
 1   rectal temperature       299 non-null    float64
 2   pulse                    299 non-null    float64
 3   pain                     299 non-null    float64
 4   abdominal distension     299 non-null    float64
 5   abdomen                  299 non-null    float64
 6   total protein            299 non-null    float64
 7   type of surgical lesion  299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.8 KB


In [19]:
data.describe()

,age,rectal temperature,pulse,pain,abdominal distension,abdomen,total protein,type of surgical lesion
count,299.00000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0
mean,1.64214,38.173244,71.297659,2.953177,2.210702,3.809365,7.155853,0.0
std,2.17730,0.655657,27.586016,1.177736,0.961901,1.170289,0.951116,0.0
min,1.00000,35.400000,30.000000,1.000000,1.000000,1.000000,3.300000,0.0
25%,1.00000,37.900000,48.000000,2.000000,1.000000,4.000000,6.600000,0.0
50%,1.00000,38.200000,64.000000,3.000000,2.000000,4.000000,7.500000,0.0
75%,1.00000,38.500000,88.000000,4.000000,3.000000,5.000000,7.500000,0.0
max,9.00000,40.800000,184.000000,5.000000,4.000000,5.000000,13.000000,0.0
